# **Image Quality experiments**

In [1]:
import sys
import torch

from tqdm import tqdm
from pyiqa import create_metric
from torchvision import transforms

sys.path.insert(0, "..\\Scripts")

c:\Users\tomma\OneDrive\Documenti\GitHub\FoodX-251_Classification\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ImageDatastore import ImageDatastore

In [3]:
batch_size = 512
transforms = transforms.Compose(
    [
        transforms.Resize(
            (232, 232), interpolation=transforms.InterpolationMode.BILINEAR
        ),
    ]
)

val_small = ImageDatastore("val_set", transform=transforms)
val_deg = ImageDatastore("val_degradate", transform=transforms)

## **Quality Estimation**

In [4]:
# Initialize NIQE metric
brisque_metric = create_metric("brisque")

Downloading: "https://huggingface.co/chaofengc/IQA-PyTorch-Weights/resolve/main/brisque_svm_weights.pth" to C:\Users\tomma\.cache\torch\hub\pyiqa\brisque_svm_weights.pth



100%|██████████| 112k/112k [00:00<00:00, 10.2MB/s]


In [5]:
# def calculate_brisque_scores(dataset):
#     """
#     Calculate brisque scores for all images in the dataset
#     """
#     brisque_metric = create_metric('brisque')
#     brisque_scores = []
    
#     for idx in range(len(dataset)):
#         image, _ = dataset[idx]
#         # Add batch dimension and ensure image is on CPU
#         image = image.unsqueeze(0).cpu()
#         # Ensure values are exactly in [0, 1] range
#         image = torch.clamp(image, 0.0, 1.0)
#         # Double check the range
#         assert image.min() >= 0.0 and image.max() <= 1.0, f"Image values out of range: min={image.min()}, max={image.max()}"
#         brisque_score = float(brisque_metric(image))
#         brisque_scores.append(brisque_score)
    
#     return brisque_scores

In [11]:
def calculate_brisque_scores(dataset):
    """
    Calculate BRISQUE scores for all images in the dataset
    """
    brisque_metric = create_metric("brisque")
    brisque_scores = []

    for idx in range(len(dataset)):
        image, _ = dataset[idx]
        # Add batch dimension and ensure image is on CPU
        image = image.unsqueeze(0).cpu()

        # Ensure correct channel ordering (B, C, H, W)
        if image.shape[1] != 3:
            image = image.permute(0, 3, 1, 2)

        # Ensure values are exactly in [0, 1] range
        image = torch.clamp(image, 0.0, 1.0)

        # Print shape for debugging
        print(f"Image shape: {image.shape}")

        try:
            brisque_score = float(brisque_metric(image))
            brisque_scores.append(brisque_score)
        except Exception as e:
            print(f"Error processing image {idx}: {str(e)}")
            print(
                f"Image stats - min: {image.min()}, max: {image.max()}, shape: {image.shape}"
            )
            raise

    return brisque_scores

In [12]:
# Initialize lists for NIQE scores
val_brisque = calculate_brisque_scores(val_small)

Image shape: torch.Size([1, 3, 232, 232])
Image shape: torch.Size([1, 3, 232, 232])
Error processing image 1: The size of tensor a (36) must match the size of tensor b (774) at non-singleton dimension 1
Image stats - min: 0.0, max: 1.0, shape: torch.Size([1, 3, 232, 232])


RuntimeError: The size of tensor a (36) must match the size of tensor b (774) at non-singleton dimension 1

In [7]:
deg_brisque = calculate_brisque_scores(val_deg)

Image 0 - min: 0.000000, max: 1.000000, shape: torch.Size([1, 3, 232, 232])
Image 1 - min: 0.000000, max: 1.000000, shape: torch.Size([1, 3, 232, 232])
Image 2 - min: 0.003801, max: 0.990090, shape: torch.Size([1, 3, 232, 232])


In [9]:
import pandas as pd

niqe_score = pd.DataFrame({'val_niqe': val_brisque, 'deg_niqe': deg_brisque})
niqe_score.describe()

,val_niqe,deg_niqe
count,11994.000000,11994.000000
mean,4.729814,6.304189
std,1.475678,2.510328
min,2.210318,2.245544
25%,3.810590,4.291391
50%,4.410388,5.682291
75%,5.260779,7.887836
max,21.560577,21.199060
